<a href="https://colab.research.google.com/github/Joyatulya/colab_notebooks/blob/main/CXR_Lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!sudo apt install -q cadaver

In [ ]:
kaggle datasets download -d raddar/chest-xrays-tuberculosis-from-india

In [ ]:
!pip install -q --upgrade pandas==1.2.5

#Enviornment Setup

In [ ]:
!git clone https://github.com/Joyatulya/utils.git
!git clone https://github.com/Joyatulya/torchvision_custom.git

In [ ]:
from utils.enviornmet_setup import *
# mount_drive()
!mkdir /root/.kaggle
config_files()

In [ ]:
!kaggle datasets download -d xhlulu/vinbigdata-chest-xray-resized-png-1024x1024
!kaggle datasets download -d raddar/tuberculosis-chest-xrays-montgomery
!kaggle datasets download -d raddar/tuberculosis-chest-xrays-shenzhen
!kaggle datasets download -d nih-chest-xrays/data

In [ ]:
!mkdir ./nih
!unzip -qq data.zip -d ./nih
!cp ./drive/MyDrive/cxr/nih/full_working_nih_df.csv ./sample/full_working_nih_df.csv
!rm data.zip
!cp ./drive/MyDrive/cxr/nih/masks_nih.zip ./masks_nih.zip
!mkdir nih
!unzip -qq masks_nih.zip -d masks_nih
!rm masks_nih.zip

!mkdir ./vinbigdata
!unzip -q vinbigdata-chest-xray-resized-png-1024x1024.zip -d ./vinbigdata
!rm vinbigdata-chest-xray-resized-png-1024x1024.zip

!cp -r ./drive/MyDrive/cxr/vinbigdata/* ./vinbigdata

!mkdir montgomery
!unzip -q tuberculosis-chest-xrays-montgomery.zip  -d ./montgomery/
!rm tuberculosis-chest-xrays-montgomery.zip

!mkdir shenzhen
!unzip -q tuberculosis-chest-xrays-shenzhen.zip -d ./shenzhen/
!rm tuberculosis-chest-xrays-shenzhen.zip

!mkdir combined
!cp -r ./drive/MyDrive/cxr/tb/* ./combined

In [ ]:

!cp ./drive/MyDrive/cxr/nih/worst_cxr.txt ./worst_cxr.txt
!cp ./drive/MyDrive/cxr/nih/last_cxr.txt ./last_cxr.txt
# !cp ./drive/MyDrive/predicted_full_nih.csv ./predicted_full_nih.csv

#Libraries & Hyper

In [ ]:
!pip install -q wandb pytorch-lightning transforms

In [ ]:
import torch
import pandas as pd
import numpy as np
import wandb
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
import matplotlib.pyplot as plt
import os
import seaborn as sns
import shutil
import pathlib
import h5py
from glob import glob
from tqdm.auto import tqdm
from sklearn import metrics
from torchvision.io import read_image, ImageReadMode

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Resize
from torch import nn
import cv2

from torch.utils import data
import pytorch_lightning as pl
pl.seed_everything(1996)
wandb.login()

In [ ]:
CHANNELS  = 3
IMG_SIZE = (300,300,CHANNELS)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
BASE_FOLDER = os.path.join('drive', 'MyDrive', 'cxr')

AUX_DATA_COLUMNS = ['age', 'position','ct_ratio', 'ct_area', 'right_cp_angle', 'right_cp_angle_direction',
                    'left_cp_angle', 'left_cp_angle_direction', 'cp_rel_position',
                    'height_width_ratio', 'area_ratio', 'vertical_span_ratio', 'Gender']

categorical_cols = ['Gender', 'position']

numerical_cols = ['ct_ratio','ct_area','right_cp_angle','left_cp_angle',
                  'cp_rel_position','height_width_ratio','area_ratio','vertical_span_ratio']

# Have removed hernia from here
PATHOLOGY_LIST = ['Cardiomegaly','Emphysema','Effusion',
                  'Pneumothorax','Mass','Edema','Consolidation',
                  'Fibrosis',]

configs = dict(
    data_flag = 'nih_full_pytorch',
    channels = CHANNELS,
    img_shape = IMG_SIZE,
    class_names = PATHOLOGY_LIST,
)

print(f"Using {device} accelaration")


#Loading and Cleaning data
---

##Accessory Setup

In [ ]:
# vin_df = pd.read_csv('./vinbigdata/train.csv')
# column_list = ['image_id', 'class_name',]
# vin_df = vin_df.loc[:,column_list]
# images_vin = {os.path.basename(x).split('.')[0]: x for x in 
#                    glob(os.path.join('./vinbigdata/', '*', '*.png'))}
# vin_df['path'] = vin_df["image_id"].map(images_vin.get)

# def get_all_disease(image_id):
#   total_disease = vin_df[vin_df.image_id == image_id].class_name.values
#   total_disease = '|'.join(total_disease)
#   return total_disease

# vin_df['findings'] = vin_df['image_id'].apply(get_all_disease)
# working_vin_df = vin_df.loc[:,['image_id','path','findings']].drop_duplicates()
# working_vin_df['masks'] = './vinbigdata/masks/' + working_vin_df['image_id'] + '_mask.png'

In [ ]:
# vin_aux_df = pd.read_csv('./vinbigdata/vinbigdata_aux.csv')
# vin_aux_df.rename(columns = {'image_index' : 'image_id'}, inplace = True)
# working_vin_df = pd.merge(working_vin_df, vin_aux_df, on = 'image_id')

In [ ]:
resize = T.Resize((384,384))
threshold = nn.Threshold(40, 0.)

In [ ]:
example = working_combined_df.iloc[np.random.randint(0,len(working_combined_df)),:]
img = read_image(example.path, ImageReadMode.RGB)
img = resize(img)
mask = read_image(example.masks, ImageReadMode.GRAY)
mask = resize(mask)
mask = threshold(mask)
fig, ax = plt.subplots(1,3,figsize = (30,8))
ax = ax.flatten()
ax[0].imshow(img[0,:], cmap = 'gray')
ax[1].imshow(mask[0,:], cmap = 'gray')
plt.show()
print(mask)

In [ ]:
def boundaries_lung(mask, margin = 20):
  # blank = torch.zeros((384,384))
  mask = read_image(mask, ImageReadMode.GRAY)
  mask = resize(mask)
  mask = threshold(mask)
  mask = mask[0,:]
  cols = mask.any(0).nonzero()
  rows = mask.any(1).nonzero()
  top = rows[0].item() - margin
  bottom = rows[-1].item() + margin + 3
  left = cols[0].item() - margin
  right = cols[-1].item() + margin
  # blank[top : bottom, left : right] = 1
  return [top, bottom, left, right]
# boundaries_lung(mask)

In [ ]:
boundaries_lung(combined_df.masks.values[0])

In [ ]:
# combined_df['top'], combined_df['bottom'], combined_df['left'], combined_df['right'] = 0,0,0,0
# combined_df

In [ ]:
values = full_df['masks'].apply(boundaries_lung)
tops, bottoms, lefts, rights = [],[],[],[]
for val in values:
  tops.append(val[0])
  bottoms.append(val[1])
  lefts.append(val[2])
  rights.append(val[3])
full_df['top'] = tops
full_df['bottom'] = bottoms
full_df['left'] = lefts
full_df['right'] = rights

In [ ]:
working_vin_df

In [ ]:
full_df.to_csv('full_working_nih_df.csv')
!cp ./full_working_nih_df.csv ./drive/MyDrive/cxr/nih/full_working_nih_df.csv

In [ ]:
# top, bottom, left, right = boundaries_lung(mask)
cxr_mask = boundaries_lung(mask)
img_masked = img * cxr_mask
# img = img[:,top : bottom, left : right]
# img = img[:,left : right, top : bottom]
plt.imshow(img_masked[0,:])

In [ ]:
all_strings = '|'.join(working_vin_df.findings).split('|')
pd.Series(all_strings).value_counts()

In [ ]:
# mont_df = pd.read_csv('./montgomery/montgomery_metadata.csv')
# images_mont = {os.path.basename(x): x for x in 
#                    glob(os.path.join('./montgomery/', 'images','images', '*.png'))}
# mont_df['path'] = mont_df['study_id'].apply(images_mont.get)
# mont_df.rename(columns = {'study_id' : 'image_id'}, inplace = True)

# shen_df = pd.read_csv('./shenzhen/shenzhen_metadata.csv')
# images_shen = {os.path.basename(x): x for x in 
#                    glob(os.path.join('./shenzhen/', 'images','images', '*.png'))}
# shen_df['path'] = shen_df['study_id'].apply(images_shen.get)
# shen_df.rename(columns = {'study_id' : 'image_id','sex' : 'gender'}, inplace = True)

# combined_df = pd.concat((shen_df,mont_df))
# combined_df.rename(columns = {'findings' : 'label'}, inplace = True)

# def parse_tb_findings(label):
#   if label == 'normal':
#     return 'No Finding'
#   findings = ['Consolidation']
#   label = str(label).lower()
#   if 'pneumot' in label:
#     findings.append('Pneumothorax')
#   if 'atelec' in label:
#     findings.append('Atelectasis')
#   if 'effusion' in label:
#     findings.append('Effusion')
#   if 'fibr' in label or 'scar' in label or 'calc' in label:
#     findings.append('Fibrosis')
#   if 'copd' in label or 'emphy' in label:
#     findings.append('Emphysema')
#   return '|'.join(findings)

# combined_df['findings'] = combined_df.label.apply(parse_tb_findings)
# combined_df.drop(labels = 'label',axis = 1, inplace=True)
# combined_df['masks'] = './combined/masks/' + combined_df['image_id'] + '_mask.png'

In [ ]:
# combined_aux_df = pd.read_csv('./combined/tb_aux.csv')
# combined_aux_df.rename(columns = {'image_index' : 'image_id'}, inplace = True)
# working_combined_df = pd.merge(combined_df, combined_aux_df, on = 'image_id')

In [ ]:
# raw_df = pd.read_csv(BASE_FOLDER + '/nih/Data_entry_2017.csv')
# raw_df = raw_df.drop('Unnamed: 11', axis = 1)
# raw_df.columns = ['image_id','findings', 'f/u', 'pt_id',
#                   'age', 'gender', 'position',
#                   'Width','Height', 'x', 'y']

# all_image_paths = {os.path.basename(x): x for x in 
                  #  glob(os.path.join('.', 'images*', '*', '*.png'))}
# raw_df['path'] = raw_df['image_index'].map(all_image_paths.get)
                   
# print('Scans found:', len(all_image_paths), ', Total Headers', raw_df.shape[0])
# raw_df['masks'] = './masks_nih/masks/' + raw_df['image_id'] + '_mask.png'
# raw_df

In [ ]:
# raw_df = pd.read_csv(BASE_FOLDER + '/nih/predicted_full_nih.csv')
# aux_df = pd.read_csv(BASE_FOLDER + '/nih/predicted_full_nih.csv')
# full_df = pd.merge(raw_df, aux_df, on = 'image_id')

In [ ]:
# aux_df = pd.read_csv('./nih/predicted_full_nih.csv')
# aux_df = aux_df.iloc[:, 2:]
# full_df = pd.merge(raw_df, aux_df, how = 'inner', on = 'image_index')

##Main Shit

In [ ]:
vin_df = pd.read_csv('./vinbigdata/full_working_vin_df.csv')
tb_df = pd.read_csv('./combined/full_working_combined_df.csv')

working_df = pd.read_csv(BASE_FOLDER + '/nih/full_working_nih_df.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('./nih', 'images*', '*', '*.png'))}
working_df['path'] = working_df['image_id'].map(all_image_paths.get)

In [ ]:
tb_df.sample(2)

,image_id,gender,age,path,findings,masks,ct_ratio,ct_area,right_cp_angle,right_cp_angle_direction,left_cp_angle,left_cp_angle_direction,cp_rel_position,height_width_ratio,area_ratio,vertical_span_ratio,top,bottom,left,right
602,CHNCXR_0603_1.png,Male,60,./shenzhen/images/images/CHNCXR_0603_1.png,Consolidation,./combined/masks/CHNCXR_0603_1.png_mask.png,0.454,0.366,58.671307,79.099295,54.415626,65.082797,1.155,0.817,3.447,1.509,35,326,5,363
695,MCUCXR_0047_0.png,Female,35,./montgomery/images/images/MCUCXR_0047_0.png,No Finding,./combined/masks/MCUCXR_0047_0.png_mask.png,0.474,0.338,50.782392,59.081264,50.948461,67.461443,1.000,0.734,1.291,0.980,15,285,3,356


In [ ]:
import time
from PIL import Image
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)
                                                                      
class CustomDataset(data.Dataset):

  def __init__(self, df):
    super().__init__()
    self.path = df.path.values
    self.image_id = df.image_id.values
    self.transform = T.Compose([
      # T.PILToTensor(),
                                T.Resize((284,284)),
                                T.RandomAutocontrast(0.2), 
                                T.RandomAffine(
                                    degrees = 15,
                                    translate =  (.09,.09),
                                    scale = (.86, 1.20),
                                    shear =  9
                                ),
                                AddGaussianNoise(0,0.013)
                                                      ])
  def __len__(self):
    return len(self.path)

  def __getitem__(self, idx):
    img_path =  self.path[idx]
    img_name = self.image_id[idx]
    # img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # img = np.expand_dims(img,0)
    # try:
      # img = torch.from_numpy(img)
    # except TypeError as e:
      # print('One Error')
      # img = torch.ones((1,284,284))
      # return img, 'error'
    img = read_image(img_path, ImageReadMode.GRAY)
    img = self.transform(img)
    return img,img_name

base = 'nih_' 
train_ds = CustomDataset(working_df)
train_dataloader = data.DataLoader(train_ds, batch_size = len(train_ds) // 16)
# img, label = next(iter(train_dataloader))
for i, (img,label) in tqdm(enumerate(train_dataloader), total = len(train_dataloader)):
  i = str(i)
  try:
    with h5py.File(f'{base + i}.h5', 'w') as f:
      f.create_dataset('img', data = img.numpy())
      f.create_dataset('name', data = label)
    shutil.copy(f'{base + i}.h5', f'./drive/MyDrive/padchest/{base}/{base + i}.h5')
  except TypeError as e:
    print(f'Error in Batch {base + i}')
    raise e
  # print("this ran too")
  os.remove(f'./{base + i}.h5')
  # os.remove(f'./{base + i}.zip')
  # shutil.rmtree(f'./{base + i}_img')

In [ ]:
# label[:2], tb_df.image_id.values[:2]
plt.imshow(img[0,0,:])
plt.show()
img_ = read_image(tb_df.path[0])
plt.imshow(img_[0])
# img_.shape

In [ ]:
def standardise_df(df):
  working_df = df.copy()
  working_df['findings'] = working_df['findings'].str.replace('Infiltration','Consolidation')
  working_df['findings'] = working_df['findings'].str.replace('Pneumonia','Consolidation')
  working_df['findings'] = working_df['findings'].str.replace('Atelectasis','Consolidation')
  working_df['findings'] = working_df['findings'].str.replace('Nodule/Mass','Mass')
  working_df['findings'] = working_df['findings'].str.replace('Nodule','Mass')
  working_df['findings'] = working_df['findings'].str.replace('Lung Opacity','Mass')
  working_df['findings'] = working_df['findings'].str.replace('Pulmonary fibrosis','Fibrosis')
  working_df['findings'] = working_df['findings'].str.replace('ILD','Fibrosis')
  working_df['findings'] = working_df['findings'].str.replace('Pleural effusion', 'Effusion')
  
  return working_df

In [ ]:
working_df = standardise_df(working_df)

In [ ]:
with open(BASE_FOLDER + '/updated_worst_cxr.txt', 'r') as f:
  worst_cxr = f.read().split('\n')
worst_cxr = worst_cxr[:-1]
worst_cxr = set(worst_cxr)
worst_cxr = np.array([*worst_cxr])
worst_cxr = worst_cxr.astype('int')

with open(BASE_FOLDER + '/updated_last_cxr.txt', 'r') as f:
  last_cxr = f.read().split('\n')
last_cxr = last_cxr[:-1]
last_cxr = set(last_cxr)
last_cxr = np.array([*last_cxr])
last_cxr = last_cxr.astype('int')

In [ ]:
# For Patients whose all the images were bad
num = 0
for sick_pt_id in worst_cxr:
  _df = working_df[working_df['pt_id'] == sick_pt_id]
  nf_cxr = _df[_df['findings'] == 'No Finding']
  f_cxr = _df[_df['findings'] != 'No Finding']
  nf_cxr_idx = nf_cxr.index.values
  all_findings = f_cxr['findings'].values
  all_findings = '|'.join(all_findings).split('|')
  all_findings = pd.Series(all_findings)
  n_findings = all_findings.size
  findings = all_findings.value_counts()
  
  # Percentage of labels out of all the findings
  perc_findings = findings / n_findings * 100

  mc_diseases = []
  for i,j in perc_findings.iteritems():
    if j > 20:
      mc_diseases.append(i)
      
  joined_disease = '|'.join(mc_diseases)
  for i in nf_cxr_idx:
    if len(mc_diseases) > 0:
      num += 1
      working_df.at[i, 'findings'] = joined_disease
    else:
      working_df.at[i, 'findings'] = 'No Finding'
print(num)

In [ ]:
# For the patients whose first no finding images were okay
i = 0
NUM_REVIEWED = 951
follow_up_patients = working_df['pt_id'].value_counts().index.values
reviewed_fu = set(follow_up_patients[:NUM_REVIEWED])
last_cxr_set = set(last_cxr)
worst_cxr_set = set(worst_cxr)
forward_fu = reviewed_fu - last_cxr_set
forward_fu = forward_fu - worst_cxr_set

for forward_pt in forward_fu:
  working_df = working_df.reset_index(drop = True)
  _df = working_df[working_df['pt_id'] == forward_pt]
  nf_cxr = _df[_df['findings'] == 'No Finding']
  idx = nf_cxr.index.values
  i += len(idx) - 1
  working_df = working_df.drop(index = idx[1:])

print(i)

In [ ]:
# For all patients whose last images were okay
i = 0
for last_pt in last_cxr:
  working_df = working_df.reset_index(drop = True)
  _df = working_df[working_df['pt_id'] == last_pt]
  nf_cxr = _df[_df['findings'] == 'No Finding']
  idx = nf_cxr.index.values[::-1]
  i += len(idx) - 1
  working_df = working_df.drop(index = idx[1:])
print(i)

#####Dropping with the help of Lung Segmentation

In [ ]:
def show_grid_img_df(df):
  fig, ax = plt.subplots(4,4, figsize = (15,15))
  ax = ax.flatten()
  print('Length of df ', len(df))
  df = df.sample(frac = 1.)
  for i in range(16):
    img = process_img(df.path.values[i])
    ax[i].imshow(img, cmap = 'gray')
    ax[i].set_xlabel([])
    ax[i].set_ylabel([])
  plt.show()

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_ctr = working_df[(working_df['ct_ratio'] < .24) & (working_df['findings'] == 'No Finding')]['position']
working_df = working_df.drop(index = nf_low_ctr.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_cta = working_df[(working_df['ct_area'] < .17) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_low_cta.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_ctr = working_df[(working_df['ct_ratio'] > .68) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_ctr.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_cta = working_df[(working_df['ct_area'] > .68) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_cta.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_lung_area = working_df[(working_df['area_ratio'] < .55) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_low_lung_area.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_lung_area = working_df[(working_df['area_ratio'] > 1.85) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_lung_area.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_vs = working_df[(working_df['vertical_span_ratio'] > 1.45) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_vs.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_vs = working_df[(working_df['vertical_span_ratio'] < .65) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_low_vs.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_cp_pos = working_df[(working_df['cp_rel_position'] < .7) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_low_cp_pos.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_cp_pos = working_df[(working_df['cp_rel_position'] > 1.3) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_cp_pos.index)

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_low_hw = working_df[(working_df['height_width_ratio'] < .55) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_low_hw.index)

In [ ]:
# working_df = working_df.reset_index(drop = True)
# nf_high_hw = working_df[(working_df['height_width_ratio'] > 1.5) & (working_df['findings'] == 'No Finding')]

In [ ]:
working_df = working_df.reset_index(drop = True)
nf_high_cp_angle =  working_df[((working_df['left_cp_angle'] > 145) | (working_df['right_cp_angle'] > 145)) & (working_df['findings'] == 'No Finding')]
working_df = working_df.drop(index = nf_high_cp_angle.index)

###Cleaning Data

While cleaning the data we have to make sure that we first replace all the relevant values and only after that we drop them or we would have to unnecessarily have to change the indices

In [ ]:
old_people = working_df[working_df['age'] > 100]
for people in old_people.itertuples():
  if people.age == 0:
    age = 47
  else:
    age = working_df[working_df['pt_id'] == people.pt_id]['age'].min()
  working_df.at[people.Index, 'age'] = age

In [ ]:
working_df['No Findings'] = working_df['findings'].apply(lambda x: 0 if 'No Finding' in x else 1)
working_df = working_df.drop(['No Findings'], axis = 1)

###Constructing dataframes of all the specific diseases separately so that I'm able to oversample them accordingly

#### I can maybe get out all the auto correlation of the different labels to see what are the chances of  a few diseasesa occuring together

In [ ]:
working_vin_df = standardise_df(vin_df)
working_tb_df = standardise_df(tb_df)

In [ ]:
for pathology in PATHOLOGY_LIST :
    working_df[pathology] = working_df['findings'].apply(lambda x: 1. if pathology in x else 0.)
    working_vin_df[pathology] = working_vin_df['findings'].apply(lambda x: 1. if pathology in x else 0.)
    working_tb_df[pathology] = working_tb_df['findings'].apply(lambda x: 1. if pathology in x else 0.)

In [ ]:
diseased_df = working_df[working_df['findings'] != 'No Finding']
diseases_dict = {}
for i in PATHOLOGY_LIST:
  _df = working_df.loc[working_df['findings'].str.contains(i, case = False)]
  diseases_dict.update({i:_df})

no_findings_df = working_df.loc[working_df['findings'] == 'No Finding']

In [ ]:
class_probabilities = []
for i in PATHOLOGY_LIST:
  class_probabilities.append(working_df.loc[:,i].sum() / len(working_df))
class_probabilities = np.array(class_probabilities, dtype = 'float32')
sns.barplot(x = PATHOLOGY_LIST, y = class_probabilities)
plt.xticks(rotation = 90)

In [ ]:
augmented_df = working_df.copy()
for disease_name, df in diseases_dict.items():
  df_len = len(df)
  if disease_name in ['Cardiomegaly','Emphysema','Edema','Fibrosis']:
    sample = df.sample(frac = 3,replace = True)
  elif disease_name in ['Pneumothorax','Mass',]:
    sample = df.sample(frac = 2.2, replace = True)
  elif disease_name in ['Effusion','Atelectasis']:
    sample = df.sample(frac = .6, replace = True)
  # else:
    # sample = df.sample(frac = 0.2, replace = True)
  augmented_df = pd.concat((augmented_df, sample), axis = 0)

In [ ]:
class_probabilities = []
for i in PATHOLOGY_LIST:
  class_probabilities.append(augmented_df.loc[:,i].sum() / len(augmented_df))
class_probabilities = np.array(class_probabilities, dtype = 'float32')
sns.barplot(x = PATHOLOGY_LIST, y = class_probabilities)
plt.xticks(rotation = 90)

In [ ]:
augmented_df = augmented_df.replace((np.inf, -np.inf), np.nan)
augmented_df = augmented_df.dropna(axis = 0)

In [ ]:
augmented_df = augmented_df[augmented_df.ct_ratio > 0]
augmented_df = augmented_df[augmented_df.ct_ratio < 10]
augmented_df = augmented_df.reset_index(drop = True)
hernia = augmented_df[augmented_df['findings'] == 'Hernia']
augmented_df = augmented_df.drop(index = hernia.index)

####Trying shit out

In [ ]:
imgs = working_df[working_df['findings'].str.contains('Consolidation')]['path'].values
fig, ax  = plt.subplots(5,5,figsize = (30,30))
ax = ax.flatten()
for i in range(25):
  img = np.random.choice(imgs)
  img = read_image(img, ImageReadMode.GRAY)
  ax[i].imshow(img[0], cmap = 'gray')

#Maybe Processing

####All the dropping

In [ ]:
follow_up_patients = working_df['pt_id'].value_counts().index.values

So that I dont forget, the following patients are the ones in which im seeing all the cases which were labeled No finding and among them I have realised that the ones in the first are the ones which have no finding and all the others are bad.

In bekar_id all the xrays seem to be bad, and I can either discard the no findings of these or change them into the findings there are in the counter part.

In last good, the last of the xrays are better and jsut have to -1 for them

Remember that even for the ones whose inital cxr are fine, we can also go and check if we can get some relevant cxr to our problem514

did cases till 550

In [ ]:
i = 922

In [ ]:
def open_cxr_file(file):
  with open(file, 'r') as f:
    open_cxr = f.read().split('\n')
  open_cxr = open_cxr[:-1]
  open_cxr = set(open_cxr)
  open_cxr = np.array([*open_cxr])
  open_cxr = open_cxr.astype('int')
  return open_cxr

In [ ]:
last_cxr_file = BASE_FOLDER + '/updated_last_cxr.txt'
with open(last_cxr_file, 'a') as f:
      f.write(str(patient_id)+'\n')
last_cxr = open_cxr_file(last_cxr_file)

In [ ]:
worst_cxr_file = BASE_FOLDER + '/updated_worst_cxr.txt'
with open(worst_cxr_file, 'a') as f:
      f.write(str(patient_id)+'\n')
worst_cxr = open_cxr_file(worst_cxr_file)

In [ ]:
well = working_df[working_df['pt_id'] == follow_up_patients[i]]
all_cxr = len(well)
no_finding_cxr = len(well[well['findings'] == 'No Finding'])
finding_cxr = well[well['findings'] != 'No Finding']['findings'].value_counts()

try:
  wellwell = well[well['findings'] == 'No Finding']['path'].values[0]
  wellwellmiddle = well[well['findings'] == 'No Finding']['path'].values[no_finding_cxr // 2]
  wellwelllast = well[well['findings'] == 'No Finding']['path'].values[-1]
  fig, ax = plt.subplots(1,3, figsize = (25,12))
  patient_id = follow_up_patients[i]
  ax[0].imshow(read_image(wellwell)[0,:], cmap = 'gray')
  ax[1].imshow(read_image(wellwellmiddle)[0,:], cmap = 'gray')
  ax[2].imshow(read_image(wellwelllast)[0,:], cmap = 'gray')
  plt.show()
  print('total cxr',len(well))
  print('normal cxr',no_finding_cxr)
  print('patient_id',patient_id)
  print('Current_index',i)
  print('Different Diseases => \n',finding_cxr)
  already = patient_id in [*worst_cxr, *last_cxr]
  print('already seen this patient') if already else None
  i += 1
except IndexError as e:
  print("No normal xrays")
  i += 1


In [ ]:
sick_pt_id = worst_cxr[np.random.randint(0,len(worst_cxr))]
_df = working_df[working_df['pt_id'] == 12880]
nf_cxr = _df[_df['findings'] == 'No Finding']
f_cxr = _df[_df['findings'] != 'No Finding']
n = len(_df)
nf_n = len(nf_cxr)
f_n = len(f_cxr)

print('Total Cxr = {}, nf_cxr = {}, f_cxr = {}, Percentage NF = {:.2f}%  '.format(n, nf_n, f_n, nf_n / n * 100) )
all_findings = f_cxr['findings'].values
all_findings = '|'.join(all_findings).split('|')
all_findings = pd.Series(all_findings)
n_findings = all_findings.size
findings = all_findings.value_counts()
perc_findings = findings / n_findings * 100
print('Total Number of Findings = ', n_findings)
pd.concat((perc_findings, findings), axis = 1)

In [ ]:
temp_df = nf_cxr.sample(frac = 1)
x = 25
fig, ax = plt.subplots(3,3,figsize = (x,x))
ax = ax.flatten()
for i in range(9):
  img = temp_df['path'].values[i]
  pos = temp_df['position'].values[i]
  finding = temp_df['findings'].values[i]
  img = process_img(img)
  ax[i].imshow(img, cmap  = 'gray')
  ax[i].set_title((pos, finding))
  ax[i].axis('off')
plt.tight_layout()

#Making Dataset

###Dataset

In [ ]:
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

random_transforms = torch.nn.Sequential(
    T.RandomAutocontrast(0.2), 
    T.RandomAffine(
        degrees = 15,
        translate =  (.09,.09),
        scale = (.86, 1.20),
        shear =  9
    ),
    AddGaussianNoise(0,0.013)
)

In [ ]:
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1.):
        super().__init__()
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

class CustomCXRDataset(Dataset):

  def __init__(self, df, transform = None, if_numerical_labels : bool = True, numeric_means = None,numeric_stds = None, if_isolate_lung = False):
    super().__init__()
    self.img_path = df.path.values
    self.if_numerical_labels = if_numerical_labels
    self.labels = torch.as_tensor(df.loc[:,PATHOLOGY_LIST].values)
    self.numeric_means = numeric_means
    self.numeric_stds = numeric_stds
    self.numeric_data = self.normalized_numeric_data(torch.Tensor(df.loc[:,numerical_cols].values))
    self.transform = transform

  def normalized_numeric_data(self, data):
    if self.numeric_means != None or self.numeric_stds != None:
      return (data - self.numeric_means) / self.numeric_stds
    means = data.mean(0)
    stds = data.std(0)
    assert len(means) == data.shape[-1]
    numeric_data = (data - means) / stds
    self.numeric_means = means
    self.numeric_stds = stds
    return numeric_data

  def __len__(self):
    return len(self.img_path)

  def __getitem__(self, idx):
    image = read_image(self.img_path[idx], ImageReadMode.GRAY).float()
    image = torch.mul(image, 1/255.)
    image = self.transform(image) if self.transform else image
    image = torch.cat([image,image,image])
    label = self.labels[idx]

    if self.if_numerical_labels == False:
      return image, label
    else:
      numeric = self.numeric_data[idx]
      return image, numeric, label

In [ ]:
from sklearn.model_selection import train_test_split
BATCH_SIZE = 32

SEED = 1005
CURRENT_DF = augmented_df

train_df, _interim_df = train_test_split(CURRENT_DF, test_size=0.15, random_state = SEED)
val_df, test_df = train_test_split(_interim_df, test_size=0.4, random_state = SEED)

train_df = pd.concat((train_df, working_vin_df, working_tb_df), axis = 0)
train_df = train_df[train_df.ct_ratio > 0.1]

train_ds = CustomCXRDataset(train_df, transform = random_transforms, if_isolate_lung=False)
val_ds = CustomCXRDataset(val_df, transform = False, numeric_means = train_ds.numeric_means, numeric_stds = train_ds.numeric_stds,if_isolate_lung=False)
test_ds = CustomCXRDataset(test_df, transform = False,numeric_means = train_ds.numeric_means, numeric_stds = train_ds.numeric_stds,if_isolate_lung=False)

train_dataloader = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True, pin_memory=False, num_workers=6, prefetch_factor=  4)
val_dataloader = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle = False,pin_memory = False,num_workers=6, prefetch_factor = 4)
test_dataloader = DataLoader(test_ds, batch_size = BATCH_SIZE, shuffle = False,num_workers=8 )

In [ ]:
imgs, numeric,labels = next(iter(train_dataloader))
fig, ax = plt.subplots(3,3,figsize = (20,20))
ax = ax.flatten()
for i in range(9):
  ax[i].imshow(imgs[i,0,:,:], cmap = 'gray')
  ax[i].set_title(numeric[i][:2])

##Model

In [ ]:
feature_extractor = models.efficientnet_b5(pretrained = True)

aux_data_nn = nn.Sequential(
    nn.Linear(len(numerical_cols), len(numerical_cols)),
    nn.SiLU(),
)

conv_net = nn.Sequential(
    feature_extractor.features,
    nn.AdaptiveAvgPool2d(1),
)
feature_extractor.classifier[-1]

In [ ]:
!pip install -qq validators
!pip install -qq timm==0.5.4
model_type = "GPUNet-D1" # select one from above
precision = "fp32" # select either fp32 of fp16 (for better performance on GPU)

gpunet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_gpunet', pretrained=True, model_type=model_type, model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
gpunet.train()

gpunet_classifier = gpunet.network[-1].net[:-1]
conv_net = nn.Sequential(
    gpunet.network[:-1],
    gpunet_classifier
)

In [ ]:
well.net[:-1]

In [ ]:
imgs = imgs.cuda()
gpunet= gpunet.cuda()
with torch.no_grad():
  preds = conv_net(imgs)
preds.shape

In [ ]:
feature_extractor = models.convnext_large(pretrained = True)

conv_net = nn.Sequential(
    feature_extractor.features,
    feature_extractor.avgpool,
    feature_extractor.classifier[:2],
    nn.Dropout(0.3)
)
feature_extractor.classifier

In [ ]:
!git clone https://github.com/d-li14/efficientnetv2.pytorch.git
!cp ./efficientnetv2.pytorch/effnetv2.py ./effnetv2.py

In [ ]:
from torchvision.models import EfficientNet_V2_M_Weights
feature_extractor = models.efficientnet_v2_m(weight = EfficientNet_V2_M_Weights.IMAGENET1K_V1)

aux_data_nn = nn.Sequential(
    nn.Linear(len(numerical_cols), len(numerical_cols)),
    nn.SiLU(),
)

conv_net = nn.Sequential(
    feature_extractor.features,
    feature_extractor.avgpool,
    feature_extractor.classifier[:-1]
)

feature_extractor.classifier

In [ ]:
class CXR_Net(nn.Module):
  def __init__(self):
    super(CXR_Net, self).__init__()
    self.conv_net = conv_net
    self.combined_seq = nn.Sequential(
        nn.Linear(1280 + len(numerical_cols), 512),
        nn.Dropout(0.2),
        nn.LeakyReLU(),
        nn.Linear(512, 512),
        nn.Dropout(0.2),
        nn.LeakyReLU(),
        nn.Linear(512, len(PATHOLOGY_LIST))
        )

  def forward(self, img, numeric):
    img_features = self.conv_net(img)
    size = img_features.shape
    img_features = img_features.view((size[0], size[1]))
    combined = torch.cat((img_features, numeric), 1)
    z = self.combined_seq(combined)
    return z

In [ ]:
pos_weights = np.sum(train_df.loc[:,PATHOLOGY_LIST].values, axis = 0) / len(train_df)
pos_weights = torch.tensor(pos_weights)
neg_weights = 1 - pos_weights
imratio = neg_weights / pos_weights
imratio = imratio.float().cuda()
imratio

In [ ]:
state_dict = torch.load('./drive/MyDrive/Models/nih_full/b4_sgd_cropped_moment')
model = CXR_Net()
model.load_state_dict(state_dict)
model = model.to(device)

In [ ]:
EPOCHS = 35
LEARNING_RATE = 2e-3
use_amp = True

model = CXR_Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = LEARNING_RATE,)
# optimizer = torch.optim.Adadelta(model.parameters(), )
loss_fn = nn.BCEWithLogitsLoss(pos_weight = imratio,reduction = 'sum').to(device)
scaler = torch.cuda.amp.grad_scaler.GradScaler(enabled=use_amp)
# lr_schedule = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.9)
lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optimizer, [13,25,], gamma=0.3)

In [ ]:
configs.update(
    {
     'epochs' : EPOCHS,
     'init_lr' : LEARNING_RATE,
     'lr_schedule' : lr_schedule.state_dict,
     'mixed_precision' : use_amp,
     'architecture' : 'effnet_v2',
     'loss' : loss_fn,
     'optimizer' : optimizer,
     'pos_weights' : imratio,
     'batch_size' : BATCH_SIZE,
    }
)

#Lightning

In [ ]:
from typing import Tuple, List, Dict

class pl_Dataset(pl.LightningDataModule):
  
  def __init__(
      self,
      data,
      extra_data : List = [],
      dataloader_kwargs : Dict = {},
      batch_size : int = 32,
      img_dims : Tuple[int, int, int] = (1,284,284),
      train_val_split : float = .15,
      val_test_split: float = .4,
      if_numerical_labels: bool = True
      ):
    super().__init__()
    self.dataloader_kwargs = dataloader_kwargs
    self.save_hyperparameters()
    self.transform = self.get_transform()
    self.train_df, _interim_df = train_test_split(self.hparams.data, test_size= self.hparams.train_val_split)
    self.val_df, self.test_df = train_test_split(_interim_df, test_size= self.hparams.val_test_split)
    self.train_df = pd.concat((self.train_df, *self.hparams.extra_data), axis = 0)

  def setup(self, stage=None):
    
    self.train_df = self.train_df[self.train_df.ct_ratio > 0.1]
    
    self.train_ds = CustomCXRDataset(self.train_df, transform = self.transform, if_numerical_labels = self.hparams.if_numerical_labels)
    self.val_ds = CustomCXRDataset(self.val_df, transform = False, numeric_means = self.train_ds.numeric_means, numeric_stds = self.train_ds.numeric_stds,if_numerical_labels =  self.hparams.if_numerical_labels)
    self.test_ds = CustomCXRDataset(self.test_df, transform = False,numeric_means = self.train_ds.numeric_means, numeric_stds = self.train_ds.numeric_stds,if_numerical_labels =  self.hparams.if_numerical_labels)

  def train_dataloader(self):
    return DataLoader(self.train_ds, shuffle = True, batch_size = self.hparams.batch_size, **self.dataloader_kwargs)

  def val_dataloader(self):
    return DataLoader(self.val_ds, shuffle = False, batch_size = self.hparams.batch_size, **self.dataloader_kwargs)

  def test_dataloader(self):
    return DataLoader(self.test_ds, shuffle = False, batch_size = self.hparams.batch_size, **self.dataloader_kwargs)

  def predict_dataloader(self):
    return DataLoader(self.test_ds,shuffle = False, batch_size = self.hparams.batch_size, **self.dataloader_kwargs)
  
  def get_transform(self):
    return T.Compose([
          T.RandomAutocontrast(0.2), 
          T.RandomAffine(
              degrees = 15,
              translate =  (.09,.09),
              scale = (.86, 1.20),
              shear =  9
          ),
          AddGaussianNoise(0,0.013)
    ])


In [ ]:
class Classifier(pl.LightningModule):
  def __init__(self,
              feaute_extractor,
              fe_top_params,lr,
                num_classes,
                num_numerical_cols,
                  train_df):
    super().__init__()
    self.save_hyperparameters()
    self.loss = self.loss_fn(train_df)
    self.classifier = nn.Sequential(
        conv_net,
        # nn.Linear(self.hparams.fe_top_params + self.hparams.num_numerical_cols, 512),
        nn.Linear(self.hparams.fe_top_params, 512),
        nn.Dropout(0.2),
        nn.LeakyReLU(),
        nn.Linear(512, 512),
        nn.Dropout(0.2),
        nn.LeakyReLU(),
        nn.Linear(512, self.hparams.num_classes)
    )

  def forward(self,x):
    # x = self.hparams.feature_extractor(x)
    return self.classifier(x)

  def training_step(self, batch, batch_idx):
    imgs, labels = batch
    preds = self(imgs)
    loss = self.loss(preds, labels)
    return loss

  def validation_step(self, batch, batch_idx):
    imgs, labels = batch
    preds = self(imgs)
    loss = self.loss(preds, labels)
    self.log(f'val_loss', loss)

  def loss_fn(self, train_df):
    pos_weights = np.sum(self.hparams.train_df.loc[:,PATHOLOGY_LIST].values, axis = 0) / len(train_df)
    pos_weights = torch.tensor(pos_weights)
    neg_weights = 1 - pos_weights
    imratio = (neg_weights / pos_weights).float()
    assert not any(torch.isnan(imratio))
    return nn.BCEWithLogitsLoss(pos_weight = imratio,reduction = 'sum')

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(), lr = self.hparams.lr)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [13,25,], gamma=0.3)
    return [optimizer], [scheduler]

In [ ]:
from torchvision.models import EfficientNet_V2_M_Weights

feature_extractor = models.efficientnet_v2_m(weight = EfficientNet_V2_M_Weights.IMAGENET1K_V1)

aux_data_nn = nn.Sequential(
    nn.Linear(len(numerical_cols), len(numerical_cols)),
    nn.SiLU(),
)

conv_net = nn.Sequential(
    feature_extractor.features,
    feature_extractor.avgpool,
    feature_extractor.classifier[:-1]
)

feature_extractor.classifier

In [ ]:
from sklearn.model_selection import train_test_split
NUMERICAL_LABELS = False

dataloader_config = dict(
    num_workers = 4,
    prefetch_factor = 4,
)

data_module = pl_Dataset(
  data = augmented_df,
  extra_data = [working_vin_df, working_tb_df],
  batch_size = 32,
  dataloader_kwargs = dataloader_config,
  img_dims  = (1,284,284),
  if_numerical_labels = NUMERICAL_LABELS,
)

model = Classifier(
  conv_net,
  fe_top_params  = 1280,
  lr = 1e-3,
  num_classes = len(PATHOLOGY_LIST),
  num_numerical_cols = len(numerical_cols),
  train_df = data_module.train_df
)

trainer = pl.Trainer(
  enable_progress_bar = True,
  max_epochs = 10,
  accelerator = 'auto',
  # check_val_every_n_epoch = 1,
  # num_sanity_val_steps = 2,
  # overfit_batches = 10,
  # auto_scale_batch_size = True,
  # auto_lr_find = True,
  # benchmark = True,
  # deterministic = True,
)

trainer.fit(model, datamodule = data_module)

In [ ]:
x = torch.tensor([1,2,3,4])
any(torch.isnan(x))

###Train Utils

In [ ]:
def print_predictions( imgs,y_pred, y_true):
  i = np.random.randint(0,y_true.shape[0] - 1)

  true_label = [disease for label, disease in zip(y_true[i], PATHOLOGY_LIST) if label]
  true_label = true_label if len(true_label) > 0 else ['No Findings']
  true_label = '|'.join(true_label)
  sorted_pred = sorted([(disease, label.item()) for label, disease in zip(y_pred[i], PATHOLOGY_LIST)], key = lambda x : x[1], reverse = True)
  pred = {disease : label.item() for label, disease in zip(y_pred[i], PATHOLOGY_LIST)}
  img = imgs[i]

  print("True Label is ", true_label)
  print("Prediction is ", sorted_pred)

  return img, pred, true_label

def calc_auc_prc(preds, true):
  true, preds = true.cpu().detach(), preds.cpu().detach()
  weighted_prc_score = metrics.average_precision_score(true, preds, average = 'weighted')
  weighted_auc_score = metrics.roc_auc_score(true, preds, average = 'weighted')
  return weighted_auc_score, weighted_prc_score

def individual_prc_scores(preds, true, job_type = 'train'):
  true, preds = true.cpu().detach(), preds.cpu().detach()
  individual_prc_score = metrics.average_precision_score(true, preds, average = None)
  individual_prc_score = {f'{job_type}_{disease}' : score for disease, score in zip(PATHOLOGY_LIST, individual_prc_score)}
  print(individual_prc_score)
  wandb.log(individual_prc_score)

In [ ]:
def save_auc(y_true, y_preds):
  fig = plt.figure(figsize = (15,15))
  for i in range(len(PATHOLOGY_LIST)):
    fpr, tpr , _ = metrics.roc_curve(y_true[:,i], y_preds[:,i])
    score = metrics.roc_auc_score(y_true[:,i], y_preds[:,i])
    plt.plot(fpr, tpr, label = f'{PATHOLOGY_LIST[i]} - {score:.4f}')
    plt.legend()
  wandb.log({'auc_curve' : fig})

def save_prc(y_true, y_preds):
  precision = {}
  recall = {}
  average_precision = {}
  fig = plt.figure(figsize =( 15,15))
  for i, disease in enumerate(PATHOLOGY_LIST):
    precision[disease], recall[disease], thresh = metrics.precision_recall_curve(y_true[:,i], y_preds[:,i])
    average_precision[disease] = metrics.average_precision_score(y_true[:,i], y_preds[:,i])
    plt.plot(recall[disease], precision[disease], label = f'{disease} - {average_precision[disease]:.4f}')
    plt.legend()
    # plt.title(f'PRC Curve - {wandb.run.name} ')
  wandb.log({'prc_curve' : fig})

def read_torch_img(file):
  img = read_image(file).float()
  img /= 255.
  resize = T.Resize(IMG_SIZE[:2])
  img = resize(img)
  return img[None,:]

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, lr_scheduler, logging_freq = 200):
  global lr
  total_samples = len(dataloader.dataset)
  model.train()
  cum_test = None
  cum_preds = None
  LOGGING_FREQ = logging_freq
  avg_loss = []
  avg_prc = []
  for batch, (img,numeric_data,true_labels) in enumerate(dataloader):
    img,numeric_data,true_labels=img.to(device),numeric_data.to(device),true_labels.to(device)
    if batch % LOGGING_FREQ == 0: batch_start = perf_counter()

    with torch.cuda.amp.autocast(enabled = use_amp):
      pred = model(img, numeric_data)
      loss = loss_fn(pred, true_labels)
    # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
    scaler.scale(loss).backward()

    # scaler.step() first unscales the gradients of the optimizer's assigned params.
    # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
    # otherwise, optimizer.step() is skipped.
    scaler.step(optimizer)

    # Updates the scale for neimgt iteration.
    scaler.update()

    optimizer.zero_grad(set_to_none = True)

    if batch % LOGGING_FREQ == 0:
      time_per_batch = (perf_counter() - batch_start)
      loss, current = loss.item(), batch * len(img)
      avg_loss.append(loss)
      wandb.log({'train_loss' : np.mean(avg_loss)}, commit = False)
      print(f"loss: {np.mean(avg_loss):>7f}  [{current:>5d}/{total_samples:>5d}], Time/Batch: {time_per_batch:.3f}")
      
      if batch % (LOGGING_FREQ*5) == 0:
        cum_preds = pred if cum_preds == None else torch.cat((cum_preds, pred), axis = 0,)
        cum_test = true_labels if cum_test == None else torch.cat((cum_test, true_labels), 0)
        try:
          auc, prc = calc_auc_prc(cum_preds, cum_test)
          individual_prc_scores(cum_preds, cum_test)
          avg_prc.append(prc)
          prc = np.mean(avg_prc)
          print(f'Weighted AUC = {auc:.4f}, Weighted PRC = {prc:.4f}')
          wandb.log({'auc' : auc, 'prc' : prc}, commit = False)
        except ValueError:
          print('Value Error, passing ...')
        except TypeError:
          print('Type Erro, passisng')

  if lr_scheduler:
    lr_scheduler.step()
    lr = lr_scheduler.get_last_lr()
  wandb.log({'lr' : lr}, commit  = False)

  

def val_loop(dataloader, model, loss_fn, save = True):
  global eval_metric
  total_samples = len(dataloader.dataset)
  model.eval()
  loss = 0

  cum_test = None
  cum_preds = None
  avg_loss = []
  for batch, (imgs,numeric_data, true_labels) in enumerate(dataloader):
    imgs,numeric_data,true_labels=imgs.to(device),numeric_data.to(device),true_labels.to(device)

    with torch.no_grad():
      preds = model(imgs, numeric_data)
      loss += loss_fn(preds, true_labels)

    cum_preds = preds if cum_preds == None else torch.cat((cum_preds, preds), axis = 0,)
    cum_test = true_labels if cum_test == None else torch.cat((cum_test, true_labels), 0)

  # Calculating Metrics
  try:
    auc, prc = calc_auc_prc(cum_preds, cum_test)
    individual_prc_scores(cum_preds, cum_test, job_type = 'val')

    print(f'Weighted AUC = {auc:.4f}, Weighted PRC = {prc:.4f} ')
    wandb.log({'val_auc' : auc, 'val_prc' : prc}, commit = False)
  except ValueError as e:
    print('Welp seems something happend')
    
  # Getting the loss
  loss = loss / (batch + 1)
  loss = loss.item()
  wandb.log({'val_loss' : loss}, commit = False)
  print(f"loss: {loss:>7f}")

  # Getting PRedictions
  img,pred,true_label = print_predictions( imgs, preds, true_labels)
  img,pred,true_label = print_predictions( imgs, preds, true_labels)
  if epoch > 5:
    table.add_data(epoch, wandb.Image(img), true_label, pred)

  if prc > eval_metric:
    eval_metric = prc
    print('Best Model Yet !!!')
    run_name = run.name
    if save : 
      print('Saving Model')
      torch.save(model.state_dict(), f'./{run_name}.pth')
      os.system(f'cp ./{run_name}.pth ./drive/MyDrive/Models/nih_full/{run_name}.pth')



def test_loop(dataloader, model):
  test_preds = torch.Tensor().to(device)
  test_labels = torch.Tensor().to(device)
  model.eval()
  for img, numeric_data,labels in dataloader:
    img,numeric_data,labels=img.to(device),numeric_data.to(device),labels.to(device)
    with torch.no_grad():

      preds = model(img,numeric_data)
      test_preds = torch.cat((test_preds, preds), 0)
      test_labels = torch.cat((test_labels, labels), 0)

  y_true, y_preds = test_labels.cpu() ,test_preds.cpu()
  save_auc(y_true, y_preds)
  save_prc(y_true, y_preds)

  # custom_cxr = wandb.Table(columns = ['image', 'predictions'])
  # TB_CXR = './drive/MyDrive/tb_ambala_cxr/'
  # cxrs = np.random.choice(os.listdir(TB_CXR), 30)
  # for cxr in cxrs:
  #   cxr = os.path.join(TB_CXR, cxr)
  #   img = read_torch_img(cxr)
  #   with torch.no_grad():
  #     img = img.cuda()
  #     pred = model(img)
  #   pred_dict = {disease:value.item() for disease, value in zip(PATHOLOGY_LIST, pred[0])}
  #   custom_cxr.add_data(wandb.Image(img[0]), pred_dict)
  # wandb.log({'custom_predictions' : custom_cxr})

###Train

In [ ]:
from time import perf_counter

with wandb.init(name = 'effnet_v2', project = 'nih_full_pytorch',group='V2',config=configs) as run:
  # wandb.watch(model, log_freq = 1000, log = 'all', log_graph = True,)
  table = wandb.Table(columns = ['epoch','image', 'label', 'prediction'], allow_mixed_types=True)
  eval_metric = 0
  lr = LEARNING_RATE
  for epoch in range(EPOCHS):
    wandb.log({'epoch' : epoch}, commit = False)
    start = perf_counter()

    print(f"Starting {epoch + 1} epoch \n-------------------------------- ")
    print('Learning Rate = ', lr)

    print('Starting Training \n---------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer, lr_schedule, 300)

    print('Validating \n---------------------')
    val_loop(val_dataloader, model, loss_fn, save = True)

    epoch_time =  perf_counter() - start

    wandb.log({'epoch_time' : epoch_time}, commit = True)
    print('Time taken => ',epoch_time)
    print('===================================\n')

  wandb.log({'predictions_table' : table})
  test_loop(test_dataloader, model)

ABle to plo these things easily now the plan is to delete the prior stuff, get a proper pipeline for interpretation and also testing all the data with a testing function at the end. Can put that in the finally section so that no matter what happens, the interpretation will be saved.

After that we can goa nd get onto the big bouys.

Also, see that the predictions made are coming out fine.


#Evaluation

In [ ]:
# model = feature_extractor.cuda()
test_preds = torch.Tensor().to(device)
test_labels = torch.Tensor().to(device)
model = model.cuda()
for img, numeric_data, labels in test_dataloader:
  with torch.no_grad():
    img, numeric_data, labels = img.to(device),numeric_data.to(device), labels.to(device)
    preds = model(img, numeric_data)
    test_preds = torch.cat((test_preds, preds), 0)
    test_labels = torch.cat((test_labels, labels), 0)


In [ ]:
y_preds, y_true = test_preds.cpu().numpy(), test_labels.cpu().numpy()

In [ ]:
i = np.random.randint(0, len(test_ds))
y_preds[i], y_true[i]

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
fig = plt.figure(figsize = (15,15))
for i in range(len(PATHOLOGY_LIST)):
  fpr, tpr , _ = roc_curve(y_true[:,i], y_preds[:,i])
  score = roc_auc_score(y_true[:,i], y_preds[:,i])
  plt.plot(fpr, tpr, label = f'{PATHOLOGY_LIST[i]} - {score}')
  plt.legend()

In [ ]:
precision = {}
recall = {}
thresh = {}
average_precision = {}
fig = plt.figure(figsize =( 15,15))
for i, disease in enumerate(PATHOLOGY_LIST):
  precision[disease], recall[disease], thresh[disease] = metrics.precision_recall_curve(y_true[:,i], y_preds[:,i])
  average_precision[disease] = metrics.average_precision_score(y_true[:,i], y_preds[:,i])
  plt.plot(recall[disease], precision[disease], label = f'{disease} - {average_precision[disease]:.4f}')
  plt.legend()
  # plt.title(f'PRC Curve - {wandb.run.name} ')
# plt.plot()
# fig

In [ ]:
func_thresh = {}
for (disease, threshold), rec, prec in zip(thresh.items(), recall.values(), precision.values()):
  print(disease, len(threshold), len(rec), len(prec))
  zipped_rec_prec = list(zip(rec, prec))
  print(max(zipped_rec_prec, key = lambda x: 2.5 * x[0] + x[1]))
  max_values = max(list(zip(rec, prec)), key = lambda x: 1.5 * x[0] + x[1])
  idx = zipped_rec_prec.index(max_values)
  func_thresh[disease] = threshold[idx]
  print(func_thresh[disease])
  print('====================================')

In [ ]:
fig = plt.figure(figsize =( 15,15))
for disease, thresholds in thresh.items():
  plt.plot(range(len(thresholds)), thresholds, label = disease)
  j = 14000
  try:
    print(disease, thresholds[j], precision[disease][j], recall[disease][j], len(thresholds))
  except IndexError:
    pass
  plt.legend()

In [ ]:
def read_torch_img(file):
  img = read_image(file).float()
  img /= 255.
  resize = T.Resize(IMG_SIZE[:2])
  img = resize(img)
  return img[None,:]

In [ ]:
!cp -r ./drive/MyDrive/cxr_segment ./cxr_segment

In [ ]:
!cp -r ./cxr_segment ./drive/MyDrive/cxr_segment

In [ ]:
from cxr_segment.segmentation_model import Segmentation_Model
from cxr_segment.feature_extraction import LungIntepretation

def get_lung_boundaries(file_path):
  segment_model = Segmentation_Model()
  mask = segment_model.get_mask_from_file(file_path)

  lung_interpret = LungIntepretation(mask)
  return lung_interpret.get_cxr_boundaries

def mask_img(img):
  mask = torch.zeros((IMG_SIZE[:2]))
  # min(top) = max(self.top[idx], 0)
  # bottom = min(self.bottom[idx], IMG_SIZE[0])
  # left = max(self.left[idx], 0)
  # right = min(self.right[idx], IMG_SIZE[0])
  top, bottom, left, right = get_lung_boundaries(cxr)
  # print(top,bottom, left, right)
  top, bottom, left, right = min(top) * 354 // 256 , max(bottom)* 414 // 256, min(left)* 324 // 256 , max(right)* 414 // 256
  mask[top: bottom, left : right] = 1
  return img *  mask

In [ ]:
from cxr_segment import get_numeric_data

In [ ]:
state_dict = torch.load('./drive/MyDrive/Models/nih_full/b5/b5_93_9prc.pth')
model = CXR_Net()
model.load_state_dict(state_dict)
model = model.to(device)

In [ ]:
TB_CXR = './drive/MyDrive/tb_ambala_cxr/'
cxrs = np.random.choice(os.listdir(TB_CXR))
cxr = os.path.join(TB_CXR, cxrs)
img = read_torch_img(cxr)
numeric = get_numeric_data(cxr)
numeric = torch.tensor(numeric, dtype = torch.float32)
numeric_means = torch.tensor([0.47789281606674194,
0.3418794870376587,
60.031524658203125,
59.270320892333984,
0.9683226346969604,
0.935834527015686,
1.206820726394653,
0.9691731333732604])

numeric_stds = torch.tensor([0.12098916620016098,
0.10514690726995468,
18.826292037963867,
18.78540802001953,
0.11661211401224136,
0.267573744058609,
2.147334337234497,
0.2273915410041809])
model.eval()
numeric = (numeric - numeric_means) / numeric_stds
plt.figure(figsize = (10,10))
plt.imshow(img[0,0,:,:], cmap = 'gray')
with torch.no_grad():
  img, numeric = img.to(device), numeric.to(device)
  pred = model(img, numeric[None,:])
pred_numpy = pred.cpu().numpy()[0]
# func_thresh_list = [func_thresh[disease] for disease in PATHOLOGY_LIST]
# np.array((PATHOLOGY_LIST, func_thresh_list, pred_numpy, pred_numpy > func_thresh_list)).T
np.array((PATHOLOGY_LIST, pred_numpy)).T, numeric